# 02_고급 기능

## 1. OpenAI 라이브러리 설치 확인

In [ ]:
# !pip install openai python-dotenv

## 2. OpenAI 정보 읽기

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv(filename='././.env')
load_dotenv(dotenv_path=dotenv_path)

openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION","").strip()

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

model=os.getenv('DEPLOYMENT_NAME')

## 3. Chain of Thought

In [ ]:
def call_openai_api(messages, max_token=100, stop=None, n=1, temperature=0.1):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_token,
        stop=stop,
        n=n,
    )
    return response

#### This prompt gets wrong answer

In [ ]:
PROMPT_ZERO_SHOT = """A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are there?
"""
messages=[
    {"role": "user", "content": PROMPT_ZERO_SHOT},
    {"role": "assistant", "content": "The answer (arabic numerals) is "},
]

response = call_openai_api(messages, temperature=0, max_token=100)

print(response['choices'][0]["message"]["content"])

#### With CoT, the answer is correct

In [ ]:
PROMPT_ZERO_SHOT_CoT = """Q: A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are
there?
A: Let's think step by step.
"""

messages=[
    {"role": "user", "content": PROMPT_ZERO_SHOT_CoT},
]

response = call_openai_api(messages, temperature=0, max_token=100)

print(response['choices'][0]["message"]["content"])

#### Can be wrong answer with few-shot learning

In [ ]:
messages=[
    {"role": "system", "content": "Generate next conversation."},
    {"role": "user", "content": "Q: Roger has 5 tennis balss. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does Roger have now?"},
    {"role": "assistant", "content": "A: The answer is 11."},
    {"role": "user", "content": "Q: A juggler can juggle 16 balls. Half of the balls are golf balls and half of the golf balls are blue. How many blue golf balls are there?"},
]

response = call_openai_api(messages, temperature=0, max_token=100)

print(response['choices'][0]["message"]["content"])

#### With few-show CoT, the answer is correct

In [ ]:
messages=[
    {"role": "user", "content": """Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"""},
    {"role": "assistant", "content": """Roger started with 5 balls.\n 2 cans of 3 tennis balls each is 6 tennis balls.\n 5 + 6 = 11. \n The answer is 11."""},
    {"role": "user", "content": """A juggler can juggle 16 balls. Half of the balls are golf balls, and half of the golf balls are blue. How many blue golf balls are there?"""},
]

response = call_openai_api(messages, temperature=0, max_token=100)

print(response['choices'][0]["message"]["content"])

## 4. SQL

### SQL 생성

In [ ]:
prompt = "### Postgres SQL tables, with their properties:\n#\n# Employee(id, name, department_id)\n# Department(id, name, address)\n# Salary_Payments(id, employee_id, amount, date)\n#\n### A query to list the names of the departments which employed more than 10 employees in the last 3 months\n\n query: "
print(prompt)

In [ ]:
response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content": prompt}],
  temperature=0,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response['choices'][0]['message']['content'])

### SQL 해석

In [ ]:
code = "Can you explain what does this code do?\n#\n# ###\n\
   Code:\n\
   SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10\n#\n#\
   Answer:\n# "
print(code)

In [ ]:
response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content": code}],
  temperature=0,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response['choices'][0]['message']['content'])

### SQL 실제 사용 사례

In [ ]:
query = '''
Given the following SQL tables, your job is to write queries given a user’s request.

CREATE TABLE Orders (
  OrderID int,
  CustomerID int,
  OrderDate datetime,
  OrderTime varchar(8),
  PRIMARY KEY (OrderID)
);

CREATE TABLE OrderDetails (
  OrderDetailID int,
  OrderID int,
  ProductID int,
  Quantity int,
  PRIMARY KEY (OrderDetailID)
);

CREATE TABLE Products (
  ProductID int,
  ProductName varchar(50),
  Category varchar(50),
  UnitPrice decimal(10, 2),
  Stock int,
  PRIMARY KEY (ProductID)
);

CREATE TABLE Customers (
  CustomerID int,
  FirstName varchar(50),
  LastName varchar(50),
  Email varchar(100),
  Phone varchar(20),
  PRIMARY KEY (CustomerID)
);
'''

In [ ]:
request1 = "Write a SQL query which computes the average total order value for all orders on 2023-04-01."

response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":query},
               {"role":"user","content": request1}],
  temperature=0,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response['choices'][0]['message']['content'])

In [ ]:
request2 = "한글로 번역:create the 5 questions to answer from tables and show the query"

response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":query},
               {"role":"user","content": request2}],
  temperature=0,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response['choices'][0]['message']['content'])

## 5. Python algorithm

### Generate problem statement

In [ ]:
response = openai.ChatCompletion.create(
    engine=model,
    messages=[
        {
            "role": "user", "content": "Generate a coding challenge problem statement with description, "
                                       "input format, output format, constraints, and example inputs and "
                                       "outputs."
        },
        {
            "role": "assistant",
            "content": "Generate a set of input/output examples with inputs and outputs that can be parsed "
                       "using ast.literal_eval() in Python. The input/output format must adhere to the "
                       "following structure:\n\n1. In: [input_1]\nOut: [output_1]\n2. In: [input_2]\nOut: ["
                       "output_2]\n...\n\nPlease ensure that 'In:' and 'Out:' keywords are only used to "
                       "denote the input and output examples in the response. Additionally, provide examples "
                       "with a variety of Python literals, including tuples, dictionaries, lists, booleans, "
                       "and numerical values. Verify that the generated statement contains the 'In:' and "
                       "'Out:' test cases before returning the result."
        }
    ],
    temperature=1,
    max_tokens=400,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

In [ ]:
problem_statement = response.choices[0].message.content.strip()
print(problem_statement)

### Generate algorithm

In [ ]:
response = openai.ChatCompletion.create(
    engine=model,
    messages=[{
        "role": "user",
        "content": f"{problem_statement}\n\nPlease provide a Python function to solve the above problem. The "
                   "generated code should be complied without any additional modification. So, no comments, "
                   "markdown, code fence, etc in the response."
    }],
    temperature=0.3,
    max_tokens=350,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

In [ ]:
solution_code = response.choices[0].message.content.strip()
print(solution_code)